Copyright 2020 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Open X-Embodiment Datasets

![](https://robotics-transformer-x.github.io/img/overview.png)

This colab helps you **visualize** the datasets in the Open X-Embodiment Dataset, explains how to **download** them and how to **train** with them.

Table of Content:

>[Open X-Embodiment Datasets](#scrollTo=EnWCKLGGaf-d)

>[Visualize Datasets](#scrollTo=29c7oLlJbWwF)

>[Download Datasets](#scrollTo=-WHN-2OrKqGo)

>[Data Loader Example](#scrollTo=IyccDsRqwtMz)

>[Interleave Multiple Datasets](#scrollTo=ekmsGRAnw3Bp)

>[Example Dataloader to produce trajectories](#scrollTo=aew258oUbamg)

>>[Demonstration of transformation from an episode to a trajectory](#scrollTo=BK4RRYkbLN5B)

>>[Combination of multiple datasets](#scrollTo=Oy89HzymQyAq)

>[Available datasets:](#scrollTo=N2Efw2aHVfSX)



# Visualize Datasets

In [36]:
!pip install tensorflow
!pip install numpy==1.25.2
!pip install tfds-nightly   # to get most up-to-date registered datasets
!pip install apache_beam


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  Using cached numpy-1.25.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.25.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.2 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available:

In [37]:
import numpy as np
import tensorflow_datasets as tfds
from PIL import Image
from IPython import display

# DATASETS = [
#     'fractal20220817_data',
#     'kuka',
#     'bridge',
#     'taco_play',
#     'jaco_play',
#     'berkeley_cable_routing',
#     'roboturk',
#     'nyu_door_opening_surprising_effectiveness',
#     'viola',
#     'berkeley_autolab_ur5',
#     'toto',
#     'language_table',
#     'columbia_cairlab_pusht_real',
#     'stanford_kuka_multimodal_dataset_converted_externally_to_rlds',
#     'nyu_rot_dataset_converted_externally_to_rlds',
#     'stanford_hydra_dataset_converted_externally_to_rlds',
#     'austin_buds_dataset_converted_externally_to_rlds',
#     'nyu_franka_play_dataset_converted_externally_to_rlds',
#     'maniskill_dataset_converted_externally_to_rlds',
#     'cmu_franka_exploration_dataset_converted_externally_to_rlds',
#     'ucsd_kitchen_dataset_converted_externally_to_rlds',
#     'ucsd_pick_and_place_dataset_converted_externally_to_rlds',
#     'austin_sailor_dataset_converted_externally_to_rlds',
#     'austin_sirius_dataset_converted_externally_to_rlds',
#     'bc_z',
#     'usc_cloth_sim_converted_externally_to_rlds',
#     'utokyo_pr2_opening_fridge_converted_externally_to_rlds',
#     'utokyo_pr2_tabletop_manipulation_converted_externally_to_rlds',
#     'utokyo_saytap_converted_externally_to_rlds',
#     'utokyo_xarm_pick_and_place_converted_externally_to_rlds',
#     'utokyo_xarm_bimanual_converted_externally_to_rlds',
#     'robo_net',
#     'berkeley_mvp_converted_externally_to_rlds',
#     'berkeley_rpt_converted_externally_to_rlds',
#     'kaist_nonprehensile_converted_externally_to_rlds',
#     'stanford_mask_vit_converted_externally_to_rlds',
#     'tokyo_u_lsmo_converted_externally_to_rlds',
#     'dlr_sara_pour_converted_externally_to_rlds',
#     'dlr_sara_grid_clamp_converted_externally_to_rlds',
#     'dlr_edan_shared_control_converted_externally_to_rlds',
#     'asu_table_top_converted_externally_to_rlds',
#     'stanford_robocook_converted_externally_to_rlds',
#     'eth_agent_affordances',
#     'imperialcollege_sawyer_wrist_cam',
#     'iamlab_cmu_pickup_insert_converted_externally_to_rlds',
#     'uiuc_d3field',
#     'utaustin_mutex',
#     'berkeley_fanuc_manipulation',
#     'cmu_play_fusion',
#     'cmu_stretch',
#     'berkeley_gnm_recon',
#     'berkeley_gnm_cory_hall',
#     'berkeley_gnm_sac_son'
# ]


def dataset2path(dataset_name):
  if dataset_name == 'robo_net':
    version = '1.0.0'
  elif dataset_name == 'language_table':
    version = '0.0.1'
  else:
    version = '0.1.0'
  return f'gs://gresearch/robotics/{dataset_name}/{version}'


def as_gif(images, path='temp.gif'):
  # Render the images as the gif:
  images[0].save(path, save_all=True, append_images=images[1:], duration=1000, loop=0)
  gif_bytes = open(path,'rb').read()
  return gif_bytes

In [38]:
#define our large replay buffer dataset
from jaxrl2.data import VariableCapacityBuffer
import gymnasium as gym

In [46]:
import tensorflow_datasets as tfds

# Set dataset path and key
dataset = 'berkeley_gnm_cory_hall'
display_key = 'image'

# Load the dataset builder
b = tfds.builder_from_directory(builder_dir=dataset2path(dataset))

# Verify the display key exists in the dataset
if display_key not in b.info.features['steps']['observation']:
    raise ValueError(
        f"The key {display_key} was not found in this dataset.\n"
        + "Please choose a different image key to display for this dataset.\n"
        + "Here is the observation spec:\n"
        + str(b.info.features['steps']['observation']))

# Initialize replay buffer
replay_buffer = None

# Load the complete dataset without splitting
ds = b.as_dataset(split='train')  # Load the entire training set

# If you need to load multiple splits, you can use:
# ds = b.as_dataset(split=['train', 'validation', 'test'])

In [40]:
# # other elements of the episode step --> this may vary for each dataset
# for elem in next(iter(episode['steps'])).items():
#   print(elem)

In [ ]:
# ds

<_PrefetchDataset element_spec={'episode_metadata': {'file_path': TensorSpec(shape=(), dtype=tf.string, name=None)}, 'steps': DatasetSpec({'action': TensorSpec(shape=(2,), dtype=tf.float64, name=None), 'action_angle': TensorSpec(shape=(3,), dtype=tf.float64, name=None), 'discount': TensorSpec(shape=(), dtype=tf.float64, name=None), 'is_first': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_last': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_terminal': TensorSpec(shape=(), dtype=tf.bool, name=None), 'language_embedding': TensorSpec(shape=(512,), dtype=tf.float32, name=None), 'language_instruction': TensorSpec(shape=(), dtype=tf.string, name=None), 'observation': {'image': TensorSpec(shape=(64, 85, 3), dtype=tf.uint8, name=None), 'position': TensorSpec(shape=(2,), dtype=tf.float64, name=None), 'state': TensorSpec(shape=(3,), dtype=tf.float64, name=None), 'yaw': TensorSpec(shape=(1,), dtype=tf.float64, name=None)}, 'reward': TensorSpec(shape=(), dtype=tf.float64, name=None)}, 

In [51]:
import itertools
from PIL import Image
import numpy as np
import gymnasium as gym

for episode in iter(ds):
    episodes_iter = iter(episode['steps'])
    # Get the first step
    prev_step = next(itertools.islice(episodes_iter, 0, 1))
    
    images = []
    cum_sum_heading = []
    pixels = []
    next_pixels = []
    vectors = []
    next_vectors = []
    delta_locations = []
    delta_actions = []
    rewards = []
    dones = []
    actions = []
    target_speed = 4.0
    
    # Initialize with first step
    observation = prev_step['observation']
    action = prev_step["action"].numpy()
    vector = [0.0, 0.0, 0.0, observation["yaw"].numpy()[0]]
    cum_sum_heading.append(observation["yaw"].numpy())
    
    # Process remaining steps
    for step in episodes_iter:
        prev_action = action
        vectors.append(vector)
        actions.append(action)
        
        next_observation = step['observation']
        action = np.array([step["action"].numpy()[1], step["action"].numpy()[0]])
        dones.append(step["is_last"].numpy() or step["is_terminal"].numpy())
        
        delta_state = np.linalg.norm(observation["state"].numpy() - next_observation["state"].numpy())
        delta_locations.append(delta_state)
        cum_sum_heading.append(step["observation"]["yaw"].numpy())
        
        # Calculate reward
        reward = 0
        if step["is_last"].numpy():
            reward = 10
        if step["is_terminal"].numpy() and not step["is_last"].numpy():
            reward = -10
        rewards.append(reward)
        
        # Calculate next vector and delta actions
        next_vector = [
            prev_action[0],
            prev_action[1],
            np.clip(np.mean(delta_locations)/target_speed, 0, 5.1),
            np.sum(cum_sum_heading)
        ]
        delta_actions.append(np.sum((prev_action[0] - action[0])**2 + 
                                  (prev_action[1] - action[1])**2))
        
        # Process images
        pixels.append(np.asarray(Image.fromarray(observation["image"].numpy()).resize((64, 64))))
        next_pixels.append(np.asarray(Image.fromarray(next_observation["image"].numpy()).resize((64, 64))))
        next_vectors.append(next_vector)
        
        # Update for next iteration
        vector = next_vector
        observation = next_observation
        prev_step = step
    
    # Initialize replay buffer if None
    if replay_buffer is None:
        image_space = gym.spaces.Box(
            low=-1.0,
            high=1.0,
            shape=tuple(pixels[0].shape),
            dtype=np.float32,
        )
        vec_space = gym.spaces.Box(
            low=-5.1,
            high=5.1,
            shape=(4,),
            dtype=np.float32,
        )
        action_space = gym.spaces.Box(
            low=np.array([-1.0, -1.0]),  # [steering, throttle/brake]
            high=np.array([1.0, 1.0]),
            dtype=np.float32
        )
        observation_space = gym.spaces.Dict({"pixels": image_space, "vector": vec_space})
        replay_buffer = VariableCapacityBuffer(observation_space, action_space)
    
    # Process images for display
    images.append(observation["image"])
    images = [Image.fromarray(image.numpy()) for image in images]
    display.Image(as_gif(images))
    
    # Calculate factors and update vectors
    target_heading = np.sum(cum_sum_heading)
    # print(vectors)
    # for  v in vectors:
    #     print(len(v),v)
    vectors = np.array(vectors)
    heading_factor = np.exp(-(vectors[:,-1] - target_heading)**2)
    speed_factor = np.exp(-(np.array(delta_locations) - target_speed)**2)
    action_factor = np.exp(-(np.array(delta_actions))**2)
    vectors[:,-1] = np.clip(vectors[:,-1]/target_heading, -5.1, 5.1)
    rewards = np.array(rewards) + speed_factor*2 + 0.1*action_factor + heading_factor
    
    # Insert into replay buffer
    for vector, next_vector, image, next_image, done, reward, action in zip(
            vectors, next_vectors, pixels, next_pixels, dones, rewards, actions):
        entry = dict(
            observations=dict(pixels=image, vector=vector),
            next_observations=dict(pixels=next_image, vector=next_vector),
            rewards=reward,
            dones=done,
            masks=1-done,
            actions=action
        )
        replay_buffer.insert(entry)
    if replay_buffer._size>int(1e4):
        break
    print(replay_buffer._size)

5359
5370
5396
5413
5436
5450
5462
5488
5497
5506
5533
5547
5557
5568
5578
5591
5601
5623
5632
5643
5653
5661
5672
5683
5698
5713
5723
5767
5785
5791
5801
5821
5842
5856
5874
5881
5892
5903
5913
5931
5941
5947
5969
5978
5988
5997
6039
6058
6067
6079
6117
6127
6136
6149
6185
6193
6203
6215
6224
6241
6269
6278
6303
6312
6327
6337
6347
6357
6367
6396
6472
6483
6493
6510
6533
6574
6836
6844
6880
6961
6962
6971
6983
7009
7040
7054
7082
7091
7109
7122
7124
7133
7155
7165
7177
7184
7220
7242
7263
7274
7306
7345
7355
7381
7408
7421
7431
7443
7451
7460
7467
7478
7500
7510
7520
7540
7571
7581
7589
7600
7610
7621
7652
7668
7679
7691
7703
7998
8013
8022
8032
8042
8051
8063
8073
8083
8115
8127
8140
8156
8169
8225
8309
8319
8338
8347
8355
8366
8379
8390
8400
8410
8531
8548
8561
8576
8585
8594
8614
8625
8635
8666
8749
8767
8777
8794
8806
8814
8825
8852
8863
8873
8914
8930
8940
8953
8962
8974
8984
8993
9003
9024
9060
9072
9078
9141
9151
9162
9174
9213
9220
9231
9247
9272
9285
9296
9306
9322
9333
9343


2025-02-16 21:42:24.959710: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


9854
9874
9888
9947
9974
9987


In [52]:
import glob
import os
import pickle


dataset_folder = os.path.join("datasets")
os.makedirs(dataset_folder, exist_ok=True)
count=len(glob.glob(f"{dataset_folder}/*.pkl"))
final_dataset_file = os.path.join(dataset_folder, f"real_robot_data_{count}.pkl")
with open(final_dataset_file, "wb") as f:
    pickle.dump(replay_buffer, f)


In [56]:
!pip uninstall -y numpy==1.25.2
!pip uninstall -y tensorflow
!pip install 'numpy > 2.0' 

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
  Using cached numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distrax 0.1.5 requires numpy<2,>=1.23.0, but you have numpy 2.0.2 which is incompatible.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.0.2 which is incompatible.
stable-baselines3 2.4.0 requires numpy<2.0,>=1.20, but you have numpy 2.0.2 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
